# Initial settings

In [1]:
!pip install pycoingecko

In [25]:
# Import packages
from pycoingecko import CoinGeckoAPI
import pandas as pd
import datetime

# Token list settings
- 可以建好多個字典，如「公鏈」、「SBF 全家桶」、「波卡生態」、「NFT 系列」等

In [18]:
# Set the token list
token_list = ['ethereum', 'binancecoin', 'cardano', 'polkadot', 'solana', 'cosmos', 'avalanche-2', 'algorand', 'fantom', 'near']
SBF_list = ['ftx-token', 'serum', 'solana', 'bonfida', 'maps', 'raydium', 'oxygen', 'sushi']
platform_list = ['binancecoin', 'ftx-token', 'okb', 'huobi-token']

# Calculating price
- 初始設定為 24HR, 7日, 30日 的漲幅變化，可調整 `n_day` 來設定

In [26]:
# CoinGecko API client
cg = CoinGeckoAPI()

# Set the dataframe
column = pd.DataFrame(columns=['項目名稱', '市值排名', '代幣價格', '24HR', '7日', '30日'])

# Set the time of 1st day, 7th day, 30th day
n_day = 30
start = datetime.timedelta(days=n_day)
day_1= (datetime.datetime.now() - start).strftime("%d-%m-%Y") #first-day price
day_last_7 =(datetime.datetime.now() - datetime.timedelta(days=7)).strftime("%d-%m-%Y") #last-one-day price
day_last_1 = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%d-%m-%Y") #last-one-day price
day_last = datetime.datetime.now().strftime("%d-%m-%Y") #last-day price

# Calculate price and PnL
for i in range(len(SBF_list)):
  values = list()
  values.append(token_list[i])
  day_last_price = cg.get_coin_history_by_id(id=token_list[i], date=day_last)['market_data']['current_price']['usd']
  day_last1_price = cg.get_coin_history_by_id(id=token_list[i], date=day_last_1)['market_data']['current_price']['usd']
  day_last7_price = cg.get_coin_history_by_id(id=token_list[i], date=day_last_7)['market_data']['current_price']['usd']
  day_1_price = cg.get_coin_history_by_id(id=token_list[i], date=day_1)['market_data']['current_price']['usd']
  pnl_1 = (day_last_price - day_last1_price)/day_last1_price * 100
  pnl_7 = (day_last_price - day_last7_price)/day_last1_price * 100
  pnl_long = (day_last_price - day_1_price)/day_1_price * 100
  market_cap_rank = cg.get_coin_by_id(token_list[i])['market_cap_rank']
  values.append(market_cap_rank)
  values.append("{0:.2f}".format(day_last_price))
  values.append("{0:.2f}%".format(pnl_1, 2))
  values.append("{0:.2f}%".format(pnl_7, 2))
  values.append("{0:.2f}%".format(pnl_long, 2))
  column.loc[i] = values

# Sort the table by Marketcap
column.sort_values(by='市值排名', ascending=True)

,項目名稱,市值排名,代幣價格,24HR,7日,30日
0,ethereum,2,1971.11,5.11%,-21.05%,-25.34%
1,binancecoin,4,296.13,13.27%,-19.24%,-13.80%
2,cardano,5,1.25,8.74%,-20.15%,-19.10%
3,polkadot,9,15.86,8.43%,-47.67%,-31.24%
4,solana,14,30.88,15.53%,-32.43%,-0.34%
7,algorand,35,0.83,12.15%,-23.88%,-14.35%
5,cosmos,44,9.64,8.07%,-40.61%,-32.68%
6,avalanche-2,50,11.44,8.37%,-24.36%,-40.87%


# Write the results to csv file
- 將上方表格存成 csv 檔，可於左側檔案區按右鍵下載

In [27]:
column.to_csv("crypto_pnl.csv")